### Polygonal ROI

* This code has been taken from: https://stackoverflow.com/questions/37099262/drawing-filled-polygon-using-mouse-events-in-open-cv-using-python/37235130. 

* I have made the following modifications to the code:

    *   Instead of taking a fixed size canvas, I have modifed it to take an RGB image.
    *   Color of the lines being drawn and the final lines being drawn has been changed.
    *   The code now saves roi masked image on the disk.


In [ ]:
import numpy as np
import cv2

In [ ]:
class PolygonDrawer(object):
    def __init__(self, window_name, image):
        self.FINAL_LINE_COLOR = (0, 255, 0)
        self.WORKING_LINE_COLOR = (0, 0, 255)
        self.image = image
        self.window_name = window_name # Name for our window
        self.done = False # Flag signalling we're done
        self.current = (0, 0) # Current position, so we can draw the line-in-progress
        self.points = [] # List of points defining our polygon


    def on_mouse(self, event, x, y, buttons, user_param):
        # Mouse callback that gets called for every mouse event (i.e. moving, clicking, etc.)

        if self.done: # Nothing more to do
            return
        if event == cv2.EVENT_MOUSEMOVE:
            # We want to be able to draw the line-in-progress, so update current mouse position
            self.current = (x, y)
        elif event == cv2.EVENT_LBUTTONDOWN:
            # Left click means adding a point at current position to the list of points
            print("Adding point #%d with position(%d,%d)" % (len(self.points), x, y))
            self.points.append((x, y))
        elif event == cv2.EVENT_RBUTTONDOWN:
            # Right click means we're done
            print("Completing polygon with %d points." % len(self.points))
            self.done = True


    def run(self):
        # Let's create our working window and set a mouse callback to handle events
        cv2.namedWindow(self.window_name, flags=cv2.WINDOW_AUTOSIZE)
        # cv2.imshow(self.window_name, np.zeros(CANVAS_SIZE, np.uint8))
        cv2.imshow(self.window_name, self.image)
        cv2.waitKey(1)
        cv2.setMouseCallback(self.window_name, self.on_mouse)

        while(not self.done):
            # This is our drawing loop, we just continuously draw new images
            # and show them in the named window
            # canvas = np.zeros(CANVAS_SIZE, np.uint8)
            canvas = self.image.copy()
            if (len(self.points) > 0):
                # Draw all the current polygon segments
                cv2.polylines(canvas, np.array([self.points]), False, self.FINAL_LINE_COLOR, thickness=4)
                # And  also show what the current segment would look like
                cv2.line(canvas, self.points[-1], self.current, self.WORKING_LINE_COLOR, thickness=4)
            # Update the window
            cv2.imshow(self.window_name, canvas)
            # And wait 50ms before next iteration (this will pump window messages meanwhile)
            if cv2.waitKey(50) == 27: # ESC hit
                self.done = True

        # User finised entering the polygon points, so let's make the final drawing
        # canvas = np.zeros(CANVAS_SIZE, np.uint8)
        canvas = self.image
        mask = np.zeros(self.image.shape[:2], np.uint8)

        # of a filled polygon
        if (len(self.points) > 0):
            # cv2.fillPoly(canvas, np.array([self.points]), self.FINAL_LINE_COLOR)
            cv2.fillPoly(mask, np.array([self.points]), (255, 255, 255))
            self.image = cv2.bitwise_and(canvas, canvas, mask=mask)
        # And show it
        cv2.imshow(self.window_name, self.image)
        cv2.waitKey()
        cv2.destroyAllWindows()
        return self.image

In [ ]:
if __name__ == "__main__":
    pd = PolygonDrawer(window_name="Draw Polygonal ROI", image=image)
    image = pd.run()
    cv2.imwrite("polygon.png", image)
    print("Polygon = %s" % pd.points)